In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
import pyspark

We configure our "SparkContext" at the start - e.g. configuration options to be passed to all worker/executor nodes

In [3]:
# NOTE - we're running in 'local' context, this could be changed later to use a resource/cluster manager e.g.YARN
# conf = pyspark.SparkConf().setMaster('yarn').setAppName('YarnSparkTest')

from pyspark.sql import SparkSession

sparkSession = SparkSession.builder.master('local[*]').appName('LocalSparkSql').getOrCreate()

In [5]:
sparkSession

Read the Macbeth file from the local filesystem and count the words on each line

In [6]:
# Update if the file is somewhere else relative to the notebook
FILEPATH = ''

# e.g. if in a hadoop context, would commonly read from HDFS or S3
#FILEPATH = "hdfs://hadoop-master:9000/user/ec2-user/"

In [7]:
airportsDf = sqlContext.read.csv(FILEPATH + "airports.csv", header=True)

# Some spark dataframe operations - count and "take"
print(airportsDf.count())
airportsDf.take(5)

322


[Row(IATA_CODE='ABE', AIRPORT='Lehigh Valley International Airport', CITY='Allentown', STATE='PA', COUNTRY='USA', LATITUDE='40.65236', LONGITUDE='-75.44040'),
 Row(IATA_CODE='ABI', AIRPORT='Abilene Regional Airport', CITY='Abilene', STATE='TX', COUNTRY='USA', LATITUDE='32.41132', LONGITUDE='-99.68190'),
 Row(IATA_CODE='ABQ', AIRPORT='Albuquerque International Sunport', CITY='Albuquerque', STATE='NM', COUNTRY='USA', LATITUDE='35.04022', LONGITUDE='-106.60919'),
 Row(IATA_CODE='ABR', AIRPORT='Aberdeen Regional Airport', CITY='Aberdeen', STATE='SD', COUNTRY='USA', LATITUDE='45.44906', LONGITUDE='-98.42183'),
 Row(IATA_CODE='ABY', AIRPORT='Southwest Georgia Regional Airport', CITY='Albany', STATE='GA', COUNTRY='USA', LATITUDE='31.53552', LONGITUDE='-84.19447')]

In [8]:
# "show" is very similar to pandas "head" - in this case the data is potentially spread around a cluster
airportsDf.show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



### Clean the dataset by changing some datatypes

In [9]:
airportsDf.filter( airportsDf['IATA_CODE'].isNull() ).count()

0

In [10]:
from pyspark.sql.types import DoubleType

print("original dtypes: ", airportsDf.dtypes)

# change datatypes from string to double
airportsDf = airportsDf.withColumn("LATITUDE", airportsDf["LATITUDE"].cast(DoubleType()))
airportsDf = airportsDf.withColumn("LONGITUDE", airportsDf["LONGITUDE"].cast(DoubleType()))

print("updated dtypes: ", airportsDf.dtypes)

original dtypes:  [('IATA_CODE', 'string'), ('AIRPORT', 'string'), ('CITY', 'string'), ('STATE', 'string'), ('COUNTRY', 'string'), ('LATITUDE', 'string'), ('LONGITUDE', 'string')]
updated dtypes:  [('IATA_CODE', 'string'), ('AIRPORT', 'string'), ('CITY', 'string'), ('STATE', 'string'), ('COUNTRY', 'string'), ('LATITUDE', 'double'), ('LONGITUDE', 'double')]


**Spark Dataframe filtering - similar, though slightly different from pandas syntax**

In [11]:
# As with all spark operations, we filter the dataframe (RDD) and produce a NEW dataframe (RDD)
tropAirports = airportsDf.filter((airportsDf["LATITUDE"] > -23) & (airportsDf["LATITUDE"] < 23))
print(tropAirports.count())
tropAirports.show(5)

12
+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      BQN|Rafael Hernández ...|  Aguadilla|   PR|    USA|18.49486| -67.12944|
|      GUM|Guam Internationa...|      Agana|   GU|    USA|13.48345|-144.79598|
|      HNL|Honolulu Internat...|   Honolulu|   HI|    USA|21.31869|-157.92241|
|      ITO|Hilo Internationa...|       Hilo|   HI|    USA|19.72026|-155.04847|
|      KOA|Kona Internationa...|Kailua/Kona|   HI|    USA|19.73877|-156.04563|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



In [12]:
# Gather all rows from the cluster
tropAirports.collect()

[Row(IATA_CODE='BQN', AIRPORT='Rafael Hernández Airport', CITY='Aguadilla', STATE='PR', COUNTRY='USA', LATITUDE=18.49486, LONGITUDE=-67.12944),
 Row(IATA_CODE='GUM', AIRPORT='Guam International Airport', CITY='Agana', STATE='GU', COUNTRY='USA', LATITUDE=13.48345, LONGITUDE=-144.79598),
 Row(IATA_CODE='HNL', AIRPORT='Honolulu International Airport', CITY='Honolulu', STATE='HI', COUNTRY='USA', LATITUDE=21.31869, LONGITUDE=-157.92241),
 Row(IATA_CODE='ITO', AIRPORT='Hilo International Airport', CITY='Hilo', STATE='HI', COUNTRY='USA', LATITUDE=19.72026, LONGITUDE=-155.04847),
 Row(IATA_CODE='KOA', AIRPORT='Kona International Airport at Keahole', CITY='Kailua/Kona', STATE='HI', COUNTRY='USA', LATITUDE=19.73877, LONGITUDE=-156.04563),
 Row(IATA_CODE='LIH', AIRPORT='Lihue Airport', CITY='Lihue', STATE='HI', COUNTRY='USA', LATITUDE=21.97598, LONGITUDE=-159.33896),
 Row(IATA_CODE='OGG', AIRPORT='Kahului Airport', CITY='Kahului', STATE='HI', COUNTRY='USA', LATITUDE=20.89865, LONGITUDE=-156.43046

In [13]:
# Write a dataframe as csv to HDFS - similar to pandas to_csv
# tropAirports.write.format("csv").save("hdfs://hadoop-master:9000/user/ec2-user/tropAirports.csv", header=True)

Once the dataframe is cut down to a manageable size, we may choose to convert to a pandas dataframe for further processing

In [14]:
pdTropAirports = tropAirports.toPandas()

# pdTropAirports is a pandas Dataframe, the original was a spark dataframe
print(type(tropAirports))
print(type(pdTropAirports))

# set IATA_CODE as the index
pdTropAirports.set_index("IATA_CODE", inplace=True)
display(pdTropAirports)

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
IATA_CODE,,,,,,
BQN,Rafael Hernández Airport,Aguadilla,PR,USA,18.49486,-67.12944
GUM,Guam International Airport,Agana,GU,USA,13.48345,-144.79598
HNL,Honolulu International Airport,Honolulu,HI,USA,21.31869,-157.92241
ITO,Hilo International Airport,Hilo,HI,USA,19.72026,-155.04847
KOA,Kona International Airport at Keahole,Kailua/Kona,HI,USA,19.73877,-156.04563
LIH,Lihue Airport,Lihue,HI,USA,21.97598,-159.33896
OGG,Kahului Airport,Kahului,HI,USA,20.89865,-156.43046
PPG,Pago Pago International Airport (Tafuna Airport),Pago Pago,AS,USA,14.33102,-170.71053
PSE,Mercedita Airport,Ponce,PR,USA,18.00830,-66.56301
